In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
run_id = '3014an7l'
run_name = "neapolitan-brownie-291"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        low_sts = get_idx_with_top_column_values('sts_score',n=n,ascending=True)
    )
    return idx_d

def print_stats(df, idx_d, key,i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('pp_truelabel_probs').iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d['training_step'], n=5)
print_interesting_text_stats(df_d['training_step'], n=5)

No label flips detected
No label flips detected

###############

random 0 

Original: the predominantly amateur cast is painful to watch , so stilted and unconvincing are the performances .
Original label 0
Unique paraphrases: 2
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
"The predominantly amateur cast is painful to watch, so stilted and unconvincing are the performances.",0.91964,0.00000,0.99936,0.50000,0,"[1, 2, 4, 5]"
"The predominantly amateur cast is painful to watch, so so stilted and unconvincing are the performances.",0.92460,-0.00497,0.99805,0.49504,0,[3]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
90,5700,1,"the predominantly amateur cast is painful to watch , so stilted and unconvincing are the performances .","The predominantly amateur cast is painful to watch, so stilted and unconvincing are the performances.",0.919636,0.91964,0.919636,0,0,0,0,0.99936,0.5,-1.63044,0.815221,1,1,1,19.3523,0.472008,0,32,13,29,13,0.453706,0.111682,0.0859146,0.00876814,0.0761002,0.0256054,0.0051851,0.0133872,6.63251e-05,0.440739,0.0184676,4.3042e-05,7.4394e-06,2,2,1,15,1,2,0,85,15,1,2,0,85,0,0,0,0,0,[0],[the],[1],[The],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]","[predominantly amateur cast is painful to watch, so stilted and unconvincing are the performances.]",1,1,1,1,False,True,False,1



###############

random 1 

Original: the script isn't very good ; not even someone as gifted as hoffman ( the actor ) can make it work .
Original label 0
Unique paraphrases: 2
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
5,300,1,"morton is , as usual , brilliant .","As usual, Morton is brilliant :",0.943739,0.93845,0.938449,1,1,0,0.00529,0.72263,0.50382,-2.16159,1.08906,2,2,2,5.11716,0.124809,0,16,2,13,2,0.184014,0.0532908,0.0440649,0.00904525,0.0342724,0.00904756,0.00161184,0.00507659,6.57104e-05,0.187619,0.00649352,4.72367e-05,2.92808e-06,4,4,1,5,1,3,0,24,5,1,2,0,24,0,0,1,0,0,"[1, 2, 3, 4, 10]","[morton is, as, .]","[0, 7, 8, 11]","[As, Morton is, :]","[5, 6, 9]","[usual,, brilliant]",3,2,4,5,True,False,False,6



###############

low_sts 4 

Original: an unflinching look at the world's dispossessed .
Original label 1
Unique paraphrases: 3
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
An unflinching look at the world's dispossessed.,0.88527,0.00000,0.99854,0.50000,0,"[1, 2, 5]"
"An unflinching look at the world's dispossessed, Mr. Williams said.",0.89084,-0.00556,0.81032,0.49549,0,[3]
An unflinching look at the world's disossessed.,0.89578,-0.01050,0.83854,0.49119,0,[4]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
65,4200,1,an unflinching look at the world's dispossessed .,An unflinching look at the world's dispossessed.,0.885271,0.88527,0.885271,1,1,0,0,0.99854,0.5,-2.9375,1.46875,1,1,1,19.3523,0.472008,0,32,13,29,13,0.453706,0.111682,0.0859146,0.00876814,0.0761002,0.0256054,0.0051851,0.0133872,6.63251e-05,0.440739,0.0184676,4.3042e-05,7.4394e-06,3,3,1,8,1,2,0,40,8,1,2,0,40,0,0,0,0,0,[0],[an],[1],[An],"[2, 3, 4, 5, 6, 7, 8, 9]",[unflinching look at the world 's dispossessed.],1,1,1,1,False,True,False,1


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(10))
print("\n#### INSERTIONS ####\n")
print(insertions.head(10))


#### REMOVALS ####

the           37
"             27
.             18
,             13
'             11
an            10
another        9
perhaps it     7
for            7
this           6
dtype: int64

#### INSERTIONS ####

The        41
''         19
An         15
,          13
It          9
Another     9
As          8
:           8
``          7
For         7
dtype: int64


Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('perhaps it', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
31,1800,2,"perhaps it's cliche to call the film 'refreshing , ' but it is . 'drumline' shows a level of young , black manhood that is funny , touching , smart and complicated .","It's cliche to call the film'refreshing', but it is : 'drumline' shows a level of young, black manhood that is funny, touching, smart and complicated.",0.958082,0.95581,0.955809,1,1,0,0.00227,0.99112,0.50225,-9.619461,4.831408,0,3,3,39.208630,0.956308,0.0,56,13,52,13,1.018046,0.127232,0.084991,0.009435,0.074442,0.042066,0.008963,0.020893,0.000070,0.848555,0.019187,0.036145,0.011078,5,4,1.0,27,1,11,0,122,26,1,11,0,115,1,0,0,0,7,"[1, 2, 8, 11, 13, 17]","[perhaps it, film, refreshing, ', .]","[0, 9, 18]","[It, film'refreshing, :]","[3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]","['s cliche to call the, ', ,, but it is, ' drumline' shows a level of young, black manhood that is funny, touching, smart and complicated.]",3,5,3,6,False,False,False,6,training_step
32,1800,3,"perhaps it's cliche to call the film 'refreshing , ' but it is . 'drumline' shows a level of young , black manhood that is funny , touching , smart and complicated .","It's cliche to call the film'refreshing', but it is. 'Drumline' shows a level of young, black manhood that is funny, touching, smart and complicated.",0.958082,0.95731,0.957305,1,1,0,0.00078,0.98472,0.50077,-9.984440,4.999863,0,6,2,35.121750,0.856628,0.0,56,13,52,13,1.028009,0.126143,0.083455,0.010378,0.071963,0.042520,0.008966,0.021273,0.000067,0.814135,0.024834,0.000025,0.000003,5,4,1.0,27,1,11,0,122,26,1,11,0,115,1,0,0,0,7,"[1, 2, 8, 11, 13, 19]","[perhaps it, film, refreshing, ', drumline]","[0, 9, 21]","[It, film'refreshing, Drumline]","[3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 17, 18, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]","['s cliche to call the, ', ,, but it is.', ' shows a level of young, black manhood that is funny, touching, smart and complicated.]",3,5,3,6,False,True,False,6,training_step
34,1800,5,"perhaps it's cliche to call the film 'refreshing , ' but it is . 'drumline' shows a level of young , black manhood that is funny , touching , smart and complicated .","It's cliche to call the film'refreshing', but it is 'drumline' shows a level of young, black manhood that is funny, touching, smart and complicated.",0.958082,0.95598,0.955979,1,1,0,0.00210,0.99069,0.50208,-7.704515,3.868315,0,12,0,36.510433,1.303944,0.0,56,13,51,13,0.961238,0.124513,0.082802,0.009532,0.072165,0.041539,0.008759,0.020768,0.000070,0.787690,0.025550,0.000026,0.000003,5,4,1.0,27,1,11,0,122,26,1,10,0,115,1,0,1,0,7,"[1, 2, 8, 11, 13, 17]","[perhaps it, film, refreshing, ', .]","[0, 9]","[It, film'refreshing]","[3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]","['s cliche to call the, ', ,, but it is, ' drumline' shows a level of young, bl